In [1]:
import numpy as np

x = np.array([
    [-1.24910365, -0.21904738],
    [-1.9628843, -0.51466743],
    [-1.38231985, -0.83929468],
    [-0.65453261, -0.37499752]
])

y = np.array([
    [0.86662341],
    [0.10482995],
    [0.22962605],
    [-0.19049319]
])

xt = np.array([
    [-1.49636334, 0.06123541]
])

px = np.array([
    [-1.24910365, -0.21904738],
    [-1.9628843, -0.51466743],
    [-1.38231985, -0.83929468],
    [-0.65453261, -0.37499752]
])

pxt = np.array([
    [-1.49636334, 0.06123541]
])

w = np.array([
    [-0.10679169],
    [-0.26786012],
    [0.99288109]
])

logtheta = np.array([0, 0, 0, -1.15])


In [2]:
from JumpGP_code_py.maximize_PD import *

In [4]:
cv = [covSum, [covSEard, covNoise]]
maximize_PD(x, y, xt, px, pxt, w, logtheta, cv, bVerbose=False)

C:\Users\yxu59\AppData\Local\miniconda3\envs\jumpGP\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2027: RuntimeWarning: divide by zero encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
C:\Users\yxu59\AppData\Local\miniconda3\envs\jumpGP\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2027: RuntimeWarning: invalid value encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
C:\Users\yxu59\AppData\Local\miniconda3\envs\jumpGP\Lib\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\yxu59\AppData\Local\miniconda3\envs\jumpGP\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\yxu59\AppData\Local\miniconda3\envs\jumpGP\Lib\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


{'x': array([[-1.24910365, -0.21904738],
        [-1.9628843 , -0.51466743],
        [-1.38231985, -0.83929468],
        [-0.65453261, -0.37499752]]),
 'y': array([[ 0.86662341],
        [ 0.10482995],
        [ 0.22962605],
        [-0.19049319]]),
 'RR': np.float64(nan),
 'fs': array([[nan]]),
 'sigma': np.float64(nan),
 'xt': array([[-1.49636334,  0.06123541]]),
 'px': array([[-1.24910365, -0.21904738],
        [-1.9628843 , -0.51466743],
        [-1.38231985, -0.83929468],
        [-0.65453261, -0.37499752]]),
 'pxt': array([[-1.49636334,  0.06123541]]),
 'nll': np.float64(nan),
 'r': array([[False],
        [False],
        [False],
        [False]]),
 'gamma': array([[nan],
        [nan],
        [nan],
        [nan]]),
 'nw': np.float64(46.48964413809032),
 'w': array([-0.22056991, -0.10773327,  0.96940314]),
 'ms': nan,
 'logtheta': array([  0.        ,   0.        , -23.38865227, -15.09661113]),
 'cv': [<function JumpGP_code_py.cov.covSum.covSum(covfunc, logtheta=None, x=None,

In [5]:
nw = np.linalg.norm(w)
w = w / nw
nIter = 100

phi_xt = np.dot(np.hstack(([1], pxt[0])), w) #phi_xt shape (1,Nt)
w = w * np.sign(phi_xt)
gx, phi_x = calculate_gx(px, w)

r = gx >= 0

In [8]:
err_flag = False
for k in range(10):
    r1 = r.flatten()
    ms = np.mean(y[r1]).item()
    try:
        logtheta = minimize(loglikelihood, logtheta, args=(covSum, [covSEard, covNoise], x[r1,:], y[r1] - ms), options={'maxiter': nIter}).x
    except:
        print(f"we fail at iteration {k}")
        err_flag = True

    K = covSum([covSEard, covNoise], logtheta, x[r1,:])
    _, Kt = covSum([covSEard, covNoise], logtheta, x[r1,:], x)
    L = cholesky(K, lower=True)
    Ly = np.linalg.solve(L, y[r1] - ms)
    LK = np.linalg.solve(L, Kt)
    fs = np.dot(LK.T, Ly) + ms
    
    sigma = np.sqrt(np.mean((y[r1] - fs[r1]) ** 2))
    
    like = norm.pdf(y, loc=fs, scale=sigma)
    RR = norm.pdf(2.5 * sigma, loc=0, scale=sigma)
    prior_z = 1 / (1 + np.exp(-0.05 * nw * gx))
    prior_z = prior_z.reshape(-1,1)
    pos_z = prior_z * like / (prior_z * like + (1 - prior_z) * RR)
    
    r = pos_z >= 0.5
    # r1 = r.flatten()
    
    def wfun(wo):
        phi_w = np.dot(phi_x, wo)
        return -np.sum(r.T * np.log(1 / (1 + np.exp(-phi_w))) + (1 - r).T * np.log(1 - 1 / (1 + np.exp(-phi_w))) )

    w_flattened = w.ravel()
    from scipy.optimize import LinearConstraint
    lc = LinearConstraint(-np.array([1, *pxt.flatten()]), ub=0)

    # constraints = {'type': 'ineq', 'fun': lambda wo: np.dot(np.array([1, *pxt.flatten()]), wo)}  # -[1, pxt]
    w_new = minimize(wfun, w_flattened, constraints=lc, options={'disp': False}).x
    # constraints = {'type': 'ineq', 'fun': lambda wo: -np.array([1, *wo])}  # -[1, pxt]
    # w_new = minimize(wfun, w_flattened, constraints=constraints, options={'disp': False}).x
    # w_new = minimize(wfun, w_flattened, bounds=[(-np.inf, np.inf)], method='SLSQP').x
    # w_new = w_new.reshape(w.shape)
    conv_crit = np.linalg.norm(w_new / np.linalg.norm(w_new) - w / np.linalg.norm(w))
    if conv_crit < 1e-3:
        break
    
    w = w_new
    nw = np.linalg.norm(w)
    w = w / nw
    gx, phi_x = calculate_gx(px, w)
    
    if err_flag:
        break

C:\Users\yxu59\AppData\Local\miniconda3\envs\jumpGP\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2027: RuntimeWarning: divide by zero encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
C:\Users\yxu59\AppData\Local\miniconda3\envs\jumpGP\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2027: RuntimeWarning: invalid value encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
C:\Users\yxu59\AppData\Local\miniconda3\envs\jumpGP\Lib\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\yxu59\AppData\Local\miniconda3\envs\jumpGP\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\yxu59\AppData\Local\miniconda3\envs\jumpGP\Lib\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [10]:
r

array([[False],
       [False],
       [False],
       [False]])